In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
# Set your API key for GroQ LLM (replace with actual key)
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')


In [4]:
from datasets import load_dataset

# Replace with the actual dataset identifier on Hugging Face
dataset = load_dataset("Talhat/Customer_IT_Support")

# View the first few rows of the dataset
dataset["train"][0]

/workspaces/codespaces-blank/TroubleGraph/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 278/278 [00:00<00:00, 60635.28 examples/s]


{'body': 'Hello Customer Support,\n\nI am writing to express concern about my HP DeskJet 3755 printer, which is malfunctioning. It encounters errors when printing wirelessly, affecting all connected devices in my home network. This has significantly disrupted my ability to manage daily tasks. Could you please assist in diagnosing and resolving this issue? Any help would be appreciated. \n\nBest regards,\n\n<name> <acc_num>\n<tel_num>',
 'answer': 'Subject: Re: Issues with HP DeskJet 3755 Printer\n\nHello <name>,\n\nThank you for reaching out regarding your HP DeskJet 3755 printer issue. To address the wireless printing problem, please try the following steps:\n\n1. Restart your printer and router to refresh the network connection.\n2. Ensure your printer’s firmware is up to date via the manufacturer’s website.\n3. Confirm that all devices are connected to the same network.\n4. Reinstall the printer drivers on your devices.\n\nIf the problem persists, please let us know so we can furthe

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a knowledge graph builder.

Extract **structured triples** (subject, relationship, object) from the input text.

Your goal is to build a knowledge graph that connects:
- **Products or Services** to the **issues** they have.
- **Issues** to **all the solutions** that address them.

### Instructions:
- Extract multi-word product or service names as nodes.
- Extract detailed issue descriptions as nodes (not just keywords).
- Extract **all full solution steps** mentioned — including **alternative fixes, workarounds, and sequential steps**.
- Use the relationship `has_issue` between a product/service and an issue.
- Use the relationship `has_solution` between an issue and **each individual solution**.
- Include technical details like OS, version numbers, error codes, hardware components, or setting names if available.
- Use complete phrases or full sentences for clarity.
- Each solution should be output as a separate triple.
- **Do NOT include any markup characters like < >, [ ], or XML/HTML tags** in your output. Use plain text only.

### Input Text:
{input}

### Output Triples (format: subject, relation, object):
""")


In [7]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_groq import ChatGroq
from langchain_community.graphs import Neo4jGraph
# Initialize GroQ LLM (assuming GroQ is supported with LangChain interface)
groq_llm = ChatGroq(temperature=0.7, model_name="meta-llama/llama-4-scout-17b-16e-instruct", api_key=GROQ_API_KEY,max_retries=10)
# Initialize the Graph Transformer
llm_transformer = LLMGraphTransformer(llm=groq_llm,allowed_nodes=['product','service','Issue','solution'],  prompt=prompt,allowed_relationships=['has_issue','has_solution'])

graph = Neo4jGraph()

/tmp/ipykernel_3938/1867942914.py:9: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [ ]:
from langchain_core.documents import Document

# Convert dataset responses into LangChain documents

documents = [Document(page_content=row['body']+"\n"+row['answer']) for row in dataset['train']]
text=dataset['train'][3]['body']+"\n"+dataset['train'][3]['answer']
documents=[Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

# Inspect the generated graph data
for i, doc in enumerate(graph_documents[:3]):  # Print first 3 examples
    print(f"Document {i+1} - Nodes: {doc.nodes}")
    print(f"Document {i+1} - Relationships: {doc.relationships}")

# Store the graph data in Neo4j
graph.add_documents(graph_documents)

In [ ]:
for i in range(10,100):
  text=dataset['train'][i]['body']+"\n"+dataset['train'][i]['answer']
  documents=[Document(page_content=text)]
  graph_documents = llm_transformer.convert_to_graph_documents(documents)
  graph.add_graph_documents(graph_documents)

In [8]:
from langchain.prompts import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher query to perform **multihop reasoning** in a graph database.
Instructions:
- Use only the provided relationship types and properties in the schema.
- The query should traverse **multiple hops** between nodes.
- Start from a specific entity (e.g., Product or Service), then navigate through related entities (e.g., Issues, Causes, Solutions).
- Ensure the query spans at least two or more relationships to extract connected information.
- Do not use any relationship types or properties that are not provided in the schema.

Schema:
{schema}

Note:
- Do not include any explanations or apologies in your responses.
- Do not respond to any questions except for constructing a Cypher statement.
- Do not include any text except the generated Cypher statement.

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)
qa_prompt = ChatPromptTemplate.from_template("""
You are a helpful support assistant using a knowledge graph of products, issues, and solutions.

Based on the question and the extracted relationships from the graph, explain the issue and its solutions in a clear and helpful way.

### Question:
{question}

### Extracted Knowledge Graph Triples:
{context}

### Guidelines:
- Rephrase technical triples into complete, helpful sentences.
- Explain each solution clearly, including steps or conditions.
- Include any relevant product/service names, error messages, version numbers, or technical details.
- Do not refer to nodes or relationships as "triples"; speak naturally.

### Answer:
""")


In [9]:
from langchain_neo4j import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=groq_llm,
    graph=graph,
    verbose=True,
    top_k=100,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=qa_prompt,
    allow_dangerous_requests=True,
)

In [10]:
chain.invoke({"query": "what are the issues in Macbook Air M1 and solutions"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH p=(:Product {id: 'Macbook Air M1'})-[:HAS_ISSUE*1..2]-(issue:Issue)-[:HAS_SOLUTION*1..2]-(solution:Solution)
RETURN DISTINCT issue.id AS issueId, solution.id AS solutionId

Full Context:
[{'issueId': 'Charger Not Functioning', 'solutionId': 'Confirm That The Charger Is Securely Plugged In'}, {'issueId': 'Charger Not Functioning', 'solutionId': 'Utilize An Outlet That Is Known To Be Operational'}, {'issueId': 'Charger Not Functioning', 'solutionId': 'Inspect The Charging Port For Any Obstructions'}, {'issueId': 'Charger Not Functioning', 'solutionId': 'Perform A System Management Controller (Smc) Reset'}, {'issueId': 'Ram Sizes', 'solutionId': '8Gb Ram'}, {'issueId': 'Ram Sizes', 'solutionId': '16Gb Ram'}, {'issueId': 'Ssd Storage Capacities', 'solutionId': '256Gb Ssd'}, {'issueId': 'Ssd Storage Capacities', 'solutionId': '2Tb Ssd'}, {'issueId': 'Color Options', 'solutionId': 'Gold'}, {'issueId': 'Color Options'

{'query': 'what are the issues in Macbook Air M1 and solutions',
 'result': "The MacBook Air M1 is a powerful and efficient laptop, but like any device, it's not immune to issues. Let's explore some common problems that users may encounter and their solutions.\n\n**Charger Not Functioning**\n\nIf your MacBook Air M1 charger is not functioning, try these steps:\n\n1. **Confirm that the charger is securely plugged in**: Ensure the charger is properly connected to both your MacBook Air and the power outlet.\n2. **Utilize an outlet that is known to be operational**: Try plugging the charger into a different outlet to rule out any electrical issues.\n3. **Inspect the charging port for any obstructions**: Check the charging port for dust, debris, or corrosion that may be preventing the charger from functioning properly.\n4. **Perform a System Management Controller (SMC) reset**: Sometimes, a simple SMC reset can resolve charging issues. To do this, press the power button, the Command+Option+